# Graph Neural Network with Pytorch Geometric

In the last decade, Deep Learning approaches (e.g. Convolutional Neural Networks and Recurrent Neural Networks) allowed to achieve unprecedented performance on a broad range of problems coming from a variety of different fields (e.g. Computer Vision and Speech Recognition). Despite the results obtained, research on DL techniques has mainly focused so far on data defined on Euclidean domains (i.e. grids). Nonetheless, in a multitude of different fields, such as: Biology, Physics, Network Science, Recommender Systems and Computer Graphics; one may have to deal with data defined on non-Euclidean domains (i.e. graphs and manifolds). The adoption of Deep Learning in these particular fields has been lagging behind until very recently, primarily since the non-Euclidean nature of data makes the definition of basic operations (such as convolution) rather elusive. Geometric Deep Learning deals in this sense with the extension of Deep Learning techniques to graph/manifold structured data.

## Resources
- https://arxiv.org/pdf/1706.02216.pdf
- https://github.com/rusty1s/pytorch_geometric
- https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8
- https://towardsdatascience.com/a-gentle-introduction-to-graph-neural-network-basics-deepwalk-and-graphsage-db5d540d50b3
- http://geometricdeeplearning.com/
- https://pytorch-geometric.readthedocs.io/en/latest/


## Graph
In Computer Science, a graph is a data structure consisting of two components, verticies and edges. A graph $G$ can be well described by the set of vertices $V$ and edges $E$ it contains.
$$ G = (V,E) $$

Edges can be directed or undirected. Vertices are usually called Nodes.
![directed graph](http://think-like-a-git.net/assets/images2/directed-graph.png) ![undirected](http://think-like-a-git.net/assets/images2/undirected-graph.png)

## Graph Neural Network
Works on Graph Structure, each node in the graph is associated with a label and we would like to predict these labels  of the nodes. The concept is presented in the [first Graph Paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.1015.7227&rep=rep1&type=pdf) where it was first introduced.
In the node classification problem setup, each node $\nu$ is characterised by its feature $x_\nu$ and associated ground truth label $t_\nu$. If we have a prtially labeled graph, we would like to use the labeled nodes to predicte the labels for the unlabeled nodes. 

For partially labeled graph $G$, the goal is to use the labeled nodes to predict the unlabeled ones. It learns to represent each node with a $d$ dimensional vector (state) $h_\nu$ with contains the information of its neighborhood. 
$$\mathbf{h}_\nu = f(\mathbf{X}_\nu, \mathbf{X}_{co[\nu]}, \mathbf{h}_{ne[\nu]}, \mathbf{X}_{ne[\nu]})$$
$$\mathbf{O}_\nu = \mathcal{g}(\mathbf{h}_\nu, \mathbf{x}_\nu)$$

where $x_\nu$, $x_{co[\nu]}$,  $h_{ne[\nu]}$, $x_{ne[\nu]}$ are the features of $\nu$, the features of its edges, the states, and the features of the nodes in the neighbourhood of $\nu$, respectively. Since we are seeking a solution for $h_\nu$, using [Banach fixed point theorm](https://en.wikipedia.org/wiki/Fixed-point_theorem), namely $\exists$ solution which $x=f(x)$.

Rewriting the above  equation as an iterative update process, which is called __message passing__ or __neighbourhood aggregation__:
$$ \mathbf{H}^{t+1} = F(\mathbf{H}^t, \mathbf{X})$$

Where $\mathbf{H}$ and $\mathbf{X}$ denote the concatenation of all $h$ and $x$. $\mathbf{H}^t$ denotes the $t$-th iteration of $\mathbf{H}$. 

$\mathcal{f}$ and $\mathcal{g}$ can be interpreted as the feedfoward neural networks. Assuming for each node $\nu$ there is target $t_\nu$ the loss can be written as follow:
$$\text{loss} = \sum_{i=1}^p(\mathbf{t}_i-\mathbf{o}_i)$$
where $p$ is the number of supervised nodes (see [Graph Neural Networks](https://arxiv.org/pdf/1812.08434.pdf) )

## GraphSAGE (Graph SAmple and aggreGatE)
![graph](images/graphsage-d.png)

Unlike embedding approaches that are based on matrix factorization, we leverage node features (e.g., text attributes, node profile information, node degrees) in order to learn an embedding function that generalizes to unseen nodes. By incorporating node features in the learning algorithm, we simultaneously learn the topological structure of each node’s neighborhood as well as the distribution of node features in the neighborhood.  While we focus on feature-rich graphs (e.g., citation data with text attributes, biological data with functional/molecular markers), our approach can also make use of structural features that are present in all graphs (e.g., node degrees).

To learn embedding for each node in an inductive way: [Inductive Representation Learning on Large Graphs](https://arxiv.org/pdf/1706.02216.pdf)
![graphsage](images/graphsage.png) 

we represent each node as an aggregation of its neighbourhood. 

$$ \mathbf{h}^k_{\mathcal{N}(\nu)} \leftarrow \text{AGGREGATE}_k ({\mathbf{h}^{k-1}_u, \forall u \in \mathcal{N}(\nu)})$$

$$ \mathbf{h}^k_\nu \leftarrow \sigma(\mathbf{W}^k . \text{CONCAT}(\mathbf{h}^{k-1}_\nu, \mathbf{h}^k_{\mathcal{N}(\nu)}))  $$

## PyTorch Geometric 

Documentation at: [Pytorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html) is a fast GPU implementation of Geometric Network using PyTorch. 


In [1]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)
# connect node 0 => 1, 1 => 0, 1=> 2 and 2 => 1
data = Data(x=x, edge_index=edge_index)

![t](https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg)

In [2]:
data

Data(x=[3, 1], edge_index=[2, 4])

### 2 dimentional label data

In [3]:
x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

![Example Graph](images/example_graph.png)

In [4]:

x = torch.tensor([[2,1], [5,6], [3,7], [12,0]], dtype=torch.float)
y = torch.tensor([0, 1, 0, 1], dtype=torch.float)

edge_index = torch.tensor([[0, 2, 1, 0, 3],
                           [3, 1, 0, 1, 2]], dtype=torch.long)
#connect node 0 => 3, 2 => 1, 1 => 0, 0 => 1, 3 => 2


data = Data(x=x, y=y, edge_index=edge_index)

In [5]:
data

Data(x=[4, 2], edge_index=[2, 5], y=[4])

## A Real-World Example — RecSys Challenge 2015
Example taken form: [hands on graph neural network](https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8), you can download the data from [recSys Challenge](https://2015.recsyschallenge.com/challenge.html).



In [6]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

#data is mixed, so use low_memory false to take more records to determine its type
df = pd.read_csv('../input/yoochoose-clicks.dat', low_memory=False, parse_dates = [1])
df.columns=['session_id', 'timestamp', 'item_id', 'category']



There are few files, yoochoose-click file which made of session_d, timestamp and the item clicked and then its category and yoochoose-buys what bought at the end of each session..

In [7]:
df.head(5)

,session_id,timestamp,item_id,category
0,1,2014-04-07 10:54:09.868000+00:00,214536500,0
1,1,2014-04-07 10:54:46.998000+00:00,214536506,0
2,1,2014-04-07 10:57:00.306000+00:00,214577561,0
3,2,2014-04-07 13:56:37.614000+00:00,214662742,0
4,2,2014-04-07 13:57:19.373000+00:00,214662742,0


In [8]:
import numpy as np
np.random.seed(42)

In [9]:
buy_df = pd.read_csv('../input/yoochoose-buys.dat', header=None, parse_dates = [1])
buy_df.columns=['session_id','timestamp','item_id','price','quantity']
buy_df.head(5)

,session_id,timestamp,item_id,price,quantity
0,420374,2014-04-06 18:44:58.314000+00:00,214537888,12462,1
1,420374,2014-04-06 18:44:58.325000+00:00,214537850,10471,1
2,281626,2014-04-06 09:40:13.032000+00:00,214535653,1883,1
3,420368,2014-04-04 06:13:28.848000+00:00,214530572,6073,1
4,420368,2014-04-04 06:13:28.858000+00:00,214835025,2617,1


In [10]:
buy_df.nunique()

session_id     509696
timestamp     1136477
item_id         19949
price             735
quantity           28
dtype: int64

In [11]:
df.nunique()

session_id     9249729
timestamp     32937844
item_id          52739
category           339
dtype: int64

In [12]:
df['valid_session'] = df.session_id.map(df.groupby('session_id')['item_id'].size() > 2)
df = df.loc[df.valid_session].drop('valid_session',axis=1)
df.nunique()

session_id     4431931
timestamp     24590088
item_id          48255
category           330
dtype: int64

In [13]:
# #randomly sample a couple of them
sampled_session_id = np.random.choice(df.session_id.unique(), 1000000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    1000000
timestamp     5546275
item_id         37353
category          259
dtype: int64

In [14]:
df.isna().sum()


session_id    0
timestamp     0
item_id       0
category      0
dtype: int64

In [15]:
# average length of session 
df.groupby('session_id')['item_id'].size().mean()

5.548274

In [16]:
from sklearn.preprocessing import LabelEncoder

item_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)
df.head()

,session_id,timestamp,item_id,category
9,3,2014-04-02 13:17:46.940000+00:00,21302,0
10,3,2014-04-02 13:26:02.515000+00:00,25022,0
11,3,2014-04-02 13:30:12.318000+00:00,29039,0
48,19,2014-04-01 20:52:12.357000+00:00,5749,0
49,19,2014-04-01 20:52:13.758000+00:00,5749,0


In [17]:
df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
9,3,2014-04-02 13:17:46.940000+00:00,21302,0,False
10,3,2014-04-02 13:26:02.515000+00:00,25022,0,False
11,3,2014-04-02 13:30:12.318000+00:00,29039,0,False
48,19,2014-04-01 20:52:12.357000+00:00,5749,0,False
49,19,2014-04-01 20:52:13.758000+00:00,5749,0,False


In [18]:
df.drop_duplicates('session_id')['label'].mean()

0.085507

In [19]:
import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root, df, transform=None, pre_transform=None):
        super(YooChooseBinaryDataset, self).__init__(root, transform, pre_transform)
        self.df = df
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    @property
    def processed_file_names(self):
        return ['../input/yoochoose_click_binary_1M_sess.dataset']

    def download(self):
        pass
    
    def process(self):
        
        data_list = []

        # process by session_id
        grouped = self.df.groupby('session_id')
        for session_id, group in tqdm(grouped):
            # to create an order of event, sess_item_id will be ordered by the click.
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id', 'timestamp']].sort_values('timestamp').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes,
                                   target_nodes], dtype=torch.long)
            x = node_features

            y = torch.FloatTensor([group.label.values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [20]:
dataset = YooChooseBinaryDataset(root='../', df=df)

In [21]:
dataset[0]

Data(x=[3, 1], edge_index=[2, 2], y=[1])

In [22]:
dataset = dataset.shuffle()
train_dataset = dataset[:800000]
val_dataset = dataset[800000:900000]
test_dataset = dataset[900000:]
len(train_dataset), len(val_dataset), len(test_dataset)

(800000, 100000, 100000)

In [23]:
from torch_geometric.loader import DataLoader
batch_size= 1024
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

c:\workspace\anaconda3\lib\site-packages\torch_geometric\deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [24]:
num_items = df.item_id.max() +1 
num_items 

37353

In [25]:
embed_dim = 128
import torch
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import SAGEConv
from torch_geometric.utils import remove_self_loops, add_self_loops

from torch_geometric.nn import GraphConv, TopKPooling, GatedGraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = SAGEConv(embed_dim, 128)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.item_embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 1)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.item_embedding(x)
        x = x.squeeze(1)        

        x = F.relu(self.conv1(x, edge_index))
        #x, edge_index, edge_attr, batch, perm, score[perm]

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)

        x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

In [26]:
device = torch.device('cuda')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss() 
# Bincary Cross Entropy Loss 

In [27]:
def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)

In [28]:
from sklearn.metrics import roc_auc_score
def evaluate(loader):
    model.eval()

    predictions = []
    labels = []

    with torch.no_grad():
        for data in loader:

            data = data.to(device)
            pred = model(data).detach().cpu().numpy()

            label = data.y.detach().cpu().numpy()
            predictions.append(pred)
            labels.append(label)

    predictions = np.hstack(predictions)
    labels = np.hstack(labels)
    
    return roc_auc_score(labels, predictions)

In [29]:
for epoch in range(5):
    loss = train()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)    
    test_acc = evaluate(test_loader)
    print('Epoch: {:03d}, Loss: {:.5f}, Train Auc: {:.5f}, Val Auc: {:.5f}, Test Auc: {:.5f}'.
          format(epoch, loss, train_acc, val_acc, test_acc))

Epoch: 000, Loss: 0.27878, Train Auc: 0.78729, Val Auc: 0.74861, Test Auc: 0.74285
Epoch: 001, Loss: 0.25288, Train Auc: 0.81834, Val Auc: 0.74752, Test Auc: 0.74284
Epoch: 002, Loss: 0.23888, Train Auc: 0.83994, Val Auc: 0.73371, Test Auc: 0.73015
Epoch: 003, Loss: 0.22283, Train Auc: 0.86637, Val Auc: 0.72615, Test Auc: 0.72222
Epoch: 004, Loss: 0.20462, Train Auc: 0.88331, Val Auc: 0.71712, Test Auc: 0.71261
